In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.insert(0, os.path.dirname(os.path.dirname(os.getcwd())))

os.environ['THEANO_FLAGS'] = "device=cuda0"

In [ ]:
import shelve
import numpy as np
import lasagne
import theano
import theano.tensor as T
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from lproc import subset, rmap
from datasets import ch14dataset as dataset
from sltools.utils import gloss2seq, seq2gloss
from sltools.nn_utils import onehot, jaccard, compute_scores
from sltools.postproc import optimize_boundaries, filter_longshort

plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['figure.facecolor'] = 'white'

In [ ]:
from experiments.hmmvsrnn_reco.a_data import durations, gloss_seqs, tmpdir, \
   train_subset, val_subset, test_subset, vocabulary
from experiments.hmmvsrnn_reco.b_preprocess import skel_feat_seqs, bgr_feat_seqs

model = "rnn"
modality = "skel"
variant = "tc3"
date = "180213"

experiment_name = model + "_" + modality + "_" + (variant + "_" if variant != "" else "") + date

In [ ]:
if modality == "skel":  # Skeleton end-to-end
    from experiments.hmmvsrnn_reco.b_preprocess import skel_feat_seqs
    feats_seqs_train = [subset(skel_feat_seqs, train_subset)]
    feats_seqs_val = [subset(skel_feat_seqs, val_subset)]
    feats_seqs_test = [subset(skel_feat_seqs, test_subset)]

elif modality == "bgr":  # BGR end-to-end
    from experiments.hmmvsrnn_reco.b_preprocess import bgr_feat_seqs
    feats_seqs_train = [subset(bgr_feat_seqs, train_subset)]
    feats_seqs_val = [subset(bgr_feat_seqs, val_subset)]

elif modality == "fusion":  # Fusion end-to-end
    from experiments.hmmvsrnn_reco.b_preprocess import skel_feat_seqs
    from experiments.hmmvsrnn_reco.b_preprocess import bgr_feat_seqs
    feats_seqs_train = [
        subset(skel_feat_seqs, train_subset),
        subset(bgr_feat_seqs, train_subset)
        ]
    feats_seqs_val = [
        subset(skel_feat_seqs, val_subset),
        subset(bgr_feat_seqs, val_subset)
        ]

elif modality == "transfer":  # Transfer
    from experiments.hmmvsrnn_reco.b_preprocess_transfer import transfer_features
    transfered_feats_seqs = transfer_features(
        "hmm_skel_180122", "hmm_skel",
        max_time=128, batch_size=16,
        encoder_kwargs={'tconv_sz': 17, 'filter_dilation': 1})
    feats_seqs_train = [subset(transfered_feats_seqs, train_subset)]
    feats_seqs_val = [subset(transfered_feats_seqs, val_subset)]

# Annotations
gloss_seqs_train = subset(gloss_seqs, train_subset)
durations_train = subset(durations, train_subset)
targets_train = rmap(lambda g, d: gloss2seq(g, d, 0),
                     gloss_seqs_train, durations_train)

gloss_seqs_val = subset(gloss_seqs, val_subset)
durations_val = subset(durations, val_subset)
targets_val = rmap(lambda g, d: gloss2seq(g, d, 0), 
                   gloss_seqs_val, durations_val)

gloss_seqs_test = subset(gloss_seqs, test_subset)
durations_test = subset(durations, test_subset)
targets_test = rmap(lambda g, d: gloss2seq(g, d, 0),
                    gloss_seqs_test, durations_test)

# Training report

In [ ]:
report = shelve.open(os.path.join(tmpdir, experiment_name))
all_batch_losses = []
all_epoch_losses = []
for i in sorted([e for e in report.keys() if e.startswith("epoch")]):
    r = report[i]
    all_batch_losses += r['batch_losses']
    all_epoch_losses.append(r['epoch_loss'])

In [ ]:
plt.figure(figsize=(12, 3))
plt.plot(np.arange(len(all_epoch_losses)), all_epoch_losses, c='red')
n_batches = len(all_batch_losses) // len(all_epoch_losses)
error = np.array([np.std(all_batch_losses[i:i+n_batches]) 
                  for i in range(0, len(all_batch_losses), n_batches)])
# plt.fill_between(np.arange(len(all_epoch_losses)), 
#                  np.maximum(0.00001, all_epoch_losses - error), 
#                  all_epoch_losses + error)
plt.yscale("log")
plt.show()
# plt.semilogy([10 ** (i - 5) for i in range(5)])

In [ ]:
print("fine-tuning started from {}".format(
    sorted([(r['val_scores']['jaccard'], e)
                     for e, r in report.items()
                     if e.startswith("epoch")
                     and "params" in r.keys()])[-1][1]))

# Find best epoch for early stopping
best_epoch = sorted([(r['val_scores']['jaccard'], e)
                     for e, r in report.items()
                     if e.startswith("epoch")
                     and "params" in r.keys()])[-1][1]

print("best epoch: {}".format(best_epoch))

epoch_report = report[best_epoch]
print(epoch_report['train_scores']['jaccard'])
print(epoch_report['train_scores']['framewise'])
print(epoch_report['val_scores']['jaccard'])
print(epoch_report['train_scores']['framewise'] - epoch_report['val_scores']['framewise'])

In [ ]:
if modality == "skel": # Skeleton end-to-end
    from experiments.hmmvsrnn_reco.c_models import skel_lstm
    model_dict = skel_lstm(feats_shape=skel_feat_seqs[0][0].shape,
                           **report['model_args'])

elif modality == "bgr":  # BGR end-to-end
    from experiments.hmmvsrnn_reco.c_models import bgr_lstm
    model_dict = bgr_lstm(feats_shape=feats_seqs_train[0][1][0].shape,
                          **report['model_args'])

elif modality == "fusion":  # Fusion end-to-end
    from experiments.hmmvsrnn_reco.c_models import fusion_lstm
    model_dict = fusion_lstm(skel_feats_shape=skel_feat_seqs[0][0].shape,
                             bgr_feats_shape=bgr_feat_seqs[0][0].shape,
                             **report['model_args'])

# Reload parameters
params = epoch_report['params']
all_layers = lasagne.layers.get_all_layers(model_dict['l_linout'])
lasagne.layers.set_all_param_values(all_layers, params)

# Compile
from sltools.models.rnn import build_predict_fn
predict_fn = build_predict_fn(model_dict, batch_size, max_time)

In [ ]:
# from sltools.models.rnn import seqs2batches
# from sltools.nn_utils import seq_hinge_loss, log_softmax

# x, y, d = next(seqs2batches(feats_seqs_train, targets_train, 
#                             batch_size=batch_size, max_time=max_time, warmup=model_dict['warmup']))

# linout, masks, feats = lasagne.layers.get_output(
#     [model_dict['l_linout'], model_dict['l_mask'], model_dict["l_feats"]])
# targets = T.imatrix()
# loss = seq_hinge_loss(linout, targets, masks=masks, delta=.3)
# g = theano.grad(loss.sum(), wrt=linout)
# print(g.eval({
#     model_dict['l_in'][0].input_var: x,
#     targets: y,
#     model_dict['l_duration'].input_var: d
# }))

# Score

In [ ]:
predictions_train = [np.argmax(p, axis=1) for p in predict_fn(feats_seqs_train)]
ji_train, framewise_train, confusion_train = compute_scores(predictions_train, targets_train, vocabulary)

predictions_val = [np.argmax(p, axis=1) for p in predict_fn(feats_seqs_val)]
ji_val, framewise_val, confusion_val = compute_scores(predictions_val, targets_val, vocabulary)

print("JI: {:.4f} / {:.4f}".format(ji_train, ji_val))
print("Accuracy: {:.4f} / {:.4f}".format(framewise_train, framewise_val))

In [ ]:
cnames = [
    '∅','vattene','vieniqui','perfetto','furbo','cheduepalle','chevuoi','daccordo',
    'seipazzo','combinato','freganiente','ok','cosatifarei','basta','prendere',
    'noncenepiu','fame','tantotempo','buonissimo','messidaccordo','sonostufo']

cmap = matplotlib.cm.viridis
cmap.set_bad(cmap(0.001))

fig, ax1 = plt.subplots(1, 1, figsize=(6, 4))
im = ax1.matshow(confusion_train / np.sum(confusion_train, axis=1, keepdims=True), 
                 interpolation='none', cmap=cmap,
                 clim=(0.001, 1), norm=LogNorm(vmin=0.001, vmax=1))
ax1.set_yticks(np.arange(0, 22, 1))
ax1.set_xticks([])
ax1.set_yticklabels(cnames)

divider = make_axes_locatable(ax1)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

fig, ax2 = plt.subplots(1, 1, figsize=(6, 4))
im = ax2.matshow(confusion_val / np.sum(confusion_val, axis=1, keepdims=True), 
                 interpolation='none', cmap='viridis',
                 clim=(0.001, 1), norm=LogNorm(vmin=0.001, vmax=1))
ax2.set_yticks(np.arange(0, 22, 1))
ax2.set_xticks([])
ax2.set_yticklabels(cnames)

divider = make_axes_locatable(ax2)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

plt.show()

# Preview prediction

In [ ]:
def preview_seq(proba, ax=None):
    # 21 distinct colors
    
    cmap = np.array([[113,204,0], [209,73,251], [243,255,52], [223,119,255], 
         [139,255,150], [255,66,189], [1,222,201], [255,77,30], 
         [0,149,225], [137,106,0], [0,43,105], [255,230,180], 
         [111,0,66], [0,113,63], [251,177,255], [56,96,0], 
         [160,218,255], [74,0,6], [255,170,172], [0,62,95], 
         [93,43,0]]) / 255
    
    ax = ax or plt.gca()
    l = np.argmax(proba, axis=1)
    for g, start, stop in seq2gloss(l):
        start = max(0, start - 1)
        stop = min(len(proba), stop + 1)
        if g == 0:
            ax.plot(np.arange(start, stop), proba[start:stop, 0], ls=':', c=cmap[0])
        else:
            ax.plot(np.arange(start, stop), proba[start:stop, g], c=cmap[g])
            ax.fill_between(np.arange(start, stop),
                            0, proba[start:stop, g],
                            facecolor=cmap[g],
                            alpha=0.3)
    ax.set_ylim((0.1, 1.05))

In [ ]:
s = 62

proba = predict_fn([[fseq[s]] for fseq in feats_seqs_val])[0]
labels = onehot(gloss2seq(gloss_seqs_val[s], durations_val[s], 0), 
                np.arange(0, 21))

f = plt.figure(figsize=(13, 2))
ax = f.add_subplot(111)
preview_seq(proba[:], ax)
plt.title("model predictions")
plt.show()
f = plt.figure(figsize=(13, .7))
ax = f.add_subplot(111)
preview_seq(labels[:] * 1.0,  ax)
plt.title("targets")
plt.show()

# print(transformations[val_subset_augmented[s]])

# Analyse errors

In [ ]:
# Distribution of errors

scores = [jaccard(onehot(l, vocabulary), onehot(p, vocabulary))
         for l, p in zip(targets_val, predictions_val)]

plt.figure()
plt.hist(scores, np.linspace(0.0, 1, 40))
plt.title("Histogram of sequence-wise JI")
plt.show()

In [ ]:
# nb of false positives out of sequence vocabulary
np.mean([len(set(p_) - set(l_)) for p_, l_ in zip(predictions_val, targets_val)], axis=0)

In [ ]:
# confusion types
cum_err = np.sum(confusion_val, axis=1) - np.diag(confusion_val)
print("false pos: {}  false neg: {}, mis-class: {}".format(
    cum_err[0], np.sum(confusion_val[1:, 0]), np.sum(cum_err[1:]) - np.sum(confusion_val[1:, 0])))

In [ ]:
# correlate error with predicted gloss duration

plt.figure(figsize=(9, 4))

prediction_accuracy = [np.sum(l[start:stop] == g) 
            for p, l in zip(predictions_val, targets_val)
            for (g, start, stop) in seq2gloss(p)
            if g != 0]
none_accuracy = [np.sum(l[start:stop] == 0)
            for p, l in zip(predictions_val, targets_val)
            for (g, start, stop) in seq2gloss(p)
            if g != 0]
gloss_d = [stop - start
           for p in predictions_val 
           for (g, start, stop) in seq2gloss(p)
           if g != 0]

scores_pred = np.zeros((int(np.ceil(max(gloss_d) / 5 + 0.0001)),))
scores_none = np.zeros((int(np.ceil(max(gloss_d) / 5 + 0.0001)),))
total_d = np.zeros((int(np.ceil(max(gloss_d) / 5 + 0.0001)),))
for vp, vn, d in zip(prediction_accuracy, none_accuracy, gloss_d):
    idx = int(d / 5)
    scores_pred[idx] += vp
    scores_none[idx] += vn
    total_d[idx] += d

plt.gca().bar(np.arange(0, int(np.ceil(max(gloss_d) + 0.0001)), 5), 
              scores_pred / total_d,
              width=5,
              alpha=.5)
plt.gca().bar(np.arange(0, int(np.ceil(max(gloss_d) + 0.0001)), 5), 
              scores_none / total_d,
              width=5,
              alpha=.5)

plt.legend(["predicted class", "non-gesture class"])
plt.xlabel("subsequence duration (based on model prediction)")
plt.ylabel("accuracy")
plt.show()

# Score

In [ ]:
# Optimize duration filter

boundaries = optimize_boundaries(targets_val, predictions_val, vocabulary, (30, 100, 301))
print("Optimal range: ", boundaries)
print("FYI the score without is: {:.4f}".format(ji_val))

In [ ]:
# Validation score

ji_after, _, _ = compute_scores(
    [filter_longshort(p, boundaries, 0) for p in predictions_val], 
    targets_val, vocabulary)
print("validation score: {:.4f}".format(ji_after))

In [ ]:
# Test score

targets_test = [gloss2seq(g_, d_, 0) for g_, d_ in zip(gloss_seqs_test, durations_test)]
predictions_test = [np.argmax(p, axis=1) for p in predict_fn(feats_seqs_test)]

ji_test, _, _ = compute_scores(
    [filter_longshort(p, boundaries, 0) for p in predictions_test], 
    targets_test, vocabulary)
print("testing score: {:.4f}".format(ji_test))

# Analyse model

## Filters

In [ ]:
from sklearn.manifold import TSNE
from sltools.tconv import TemporalConv

tc_l = None
layers = lasagne.layers.get_all_layers(model_dict['l_linout'])

for l in layers:
    if isinstance(l, TemporalConv):
        tc_l = l
        break

W = np.asarray(tc_l.W.eval())
tsne = TSNE(n_components=1, n_iter=5000, n_iter_without_progress=100, verbose=True)
filter_order = np.argsort(tsne.fit_transform(W)[:, 0])

In [ ]:
ax = None
n = int(np.ceil(len(filter_order) / 200))
for i in range(0, n):
    if i % 4 == 0:
        f = plt.figure(figsize=(15, 8))
    ax = plt.subplot(1, 4, i % 4 + 1, sharey=ax)
    ax.pcolor(W[filter_order[i * 200:(i + 1) * 200], ::-1], 
              clim=(-np.abs(W).max(), np.abs(W).max()), 
              cmap='RdBu')
    
    if ((i + 1) % 4 == 0) or i + 1 == n:
        plt.show()

In [ ]:
# Show activated filters for a category

act_l = layers[layers.index(tc_l) + 2]
X = feat_seqs_val
X = rmap(lambda x_: (x_,), X)
y = [gloss2seq(g_, len(r_), 0)
     for g_, r_ in zip(gloss_seqs_val, feat_seqs_val)]

# Chunking
step = recognizer.max_len - 2 * recognizer.warmup
durations = [len(seq[0]) for seq in X]
chunks = [(i, k, min(k + recognizer.max_len, d))
          for i, d in enumerate(durations)
          for k in range(0, d - recognizer.warmup, step)]
grads = [np.zeros((d, tc_l.output_shape[2]), dtype=theano.config.floatX)
         for d in durations]

# Functions
X_buffers = [np.zeros(shape=(recognizer.batch_size, recognizer.max_len) + shape,
                      dtype=theano.config.floatX)
             for shape in recognizer.input_shapes]
y_buffer = np.zeros(shape=(recognizer.batch_size, recognizer.max_len), dtype=np.int32)
d_buffer = np.zeros((recognizer.batch_size,), dtype=np.int32)
c_buffer = np.zeros((recognizer.batch_size, 3), dtype=np.int32)
tgt_var = T.imatrix()
  
activations, predictions = lasagne.layers.get_output(
    [act_l, recognizer.l_raw], deterministic=True)
g = theano.grad(predictions[T.arange(recognizer.batch_size)[:, None], :, tgt_var].sum(), 
                wrt=activations)
g_fn = theano.function([recognizer.l_in[0].input_var, recognizer.durations_var, tgt_var], g)

j = 0
for i, (seq, start, stop) in enumerate(chunks):
    for b, x in zip(X_buffers, X[seq]):
        b[j][:stop - start] = x[start:stop]
    y_buffer[j][:stop - start] = y[seq][start:stop]
    d_buffer[j] = stop - start
    c_buffer[j] = (seq, start, stop)

    if j + 1 == recognizer.batch_size or i == len(chunks) - 1:
        batch_predictions = g_fn(*X_buffers, d_buffer, y_buffer)[:j + 1]
        for (seq_, start_, stop_), grad in zip(c_buffer, batch_predictions):
            warmup = recognizer.warmup if start_ > 0 else 0
            grads[seq_][start_ + warmup:stop_] = \
                grad[warmup:stop_ - start_]

    j = (j + 1) % recognizer.batch_size
    
all_grads = np.concatenate(grads)
all_labels = np.concatenate(y)

In [ ]:
rng = .5 # max(-X.min(), X.max())

ashes = []
for l in range(recognizer.nlabels):
    where = (all_labels == l)
    h = np.stack([np.histogram(X[where, i], bins=np.linspace(-rng, rng, 16))[0] / where.sum()
                  for i in range(X.shape[1])])
    ashes.append(h)

meanh = np.mean(ashes, axis=0)

In [ ]:
fig = plt.figure(figsize=(15, 50))

ax = plt.subplot2grid((1, 6), (0, 0), colspan=2)
ax.pcolormesh(W[filter_order, :], clim=(-np.abs(W).max(), np.abs(W).max()), cmap='bwr')
ax.set_yticks(np.arange(0, W.shape[0], 5))
# ax.set_yticks([])
ax.set_yticklabels([])
ax.grid(True)

for p, i in enumerate([0, 1, 2, -1]):
    h = ashes[i] - meanh
#     h = meanh
    ax = plt.subplot2grid((1, 6), (0, 2 + p), colspan=1)
    ax.pcolormesh(h, clim=(-1, 1), cmap='bwr')
    ax.set_yticks(np.arange(0, W.shape[0], 5))
    ax.set_xticks(np.arange(0, 16, 3))
    ax.set_xticklabels(np.linspace(-rng, rng, 6))
    ax.grid(True)
    ax.set_ylim((0, W.shape[0]))
    ax.set_yticklabels([])

fig.subplots_adjust(hspace=0, wspace=0.1)

In [ ]:
plt.figure()
plt.imshow(np.concatenate(feat_seqs[0][1280], axis=1), clim=(0, 1), cmap='gray')
plt.show()

In [ ]:
max([stop - start for gseq in gloss_seqs_val for g, start, stop in gseq])